In [ ]:
from datetime import datetime
import torch
import requests
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline, ControlNetModel, StableDiffusionControlNetPipeline

# cnmodel = "lllyasviel/sd-controlnet-openpose"
# controlnet = ControlNetModel.from_pretrained(
#     cnmodel,
#     torch_dtype=torch.float16,
#     use_auth_token=TOKEN,
# )
controlnet = [
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose", torch_dtype=torch.float16),
    ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-seg", torch_dtype=torch.float16),
]

device = "cuda"
i2ipl = "Lykon/DreamShaper"
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    i2ipl,
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16,
    # use_auth_token=TOKEN,
).to(device)

In [ ]:
from controlnet_aux import OpenposeDetector
openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")

In [ ]:
file_path = "pic/pose4.png"
init_image = Image.open(file_path).convert("RGB")
openpose_image = openpose(init_image)
openpose_image

In [ ]:
file_path = "pic/1000_F_268542308_zVVNS2DsAaVd2hJix6FZ1BjVbQqMqqCg.jpg"
init_image_bg = Image.open(file_path).convert("RGB")
init_image_bg = init_image_bg.resize(init_image.size)
init_image_bg

In [ ]:
prompt = "a vicious Pirate on ship, sitting on deck, black long hair, having an falchion, in ancient Roman, highly detailed, 4k, 8k, epic moment, MtG"
negative_prompt = "monochrome, asian, closeup, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
generator = torch.Generator(device=device).manual_seed(1024)
image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=[openpose_image, init_image_bg], controlnet_conditioning_scale=[0.9, 0.4], num_inference_steps=50, generator=generator).images[0]
now = datetime.now()
formatted_time = now.strftime("%Y%m%d%H%M%S")
image.save(f'pic/{formatted_time}.png')
image